In [1]:
import itertools
import os

import numpy as np
from scipy import sparse
from tqdm import tqdm_notebook

In [ ]:
STBM_PATH = os.path.join(settings.BEHAVIOUR_PATH, 'stbm')

In [2]:
def stbm(n_nodes=100, n_clusters=40,  # Network parameters
         n_topics=5, n_documents=30, n_slots=140, vocabulary_size=50,  # Language parameters
         ρ=None, π=None, θ=None):
    """Simulate a network + document scenario with STBM."""

    α = np.ones(n_topics)
    β = np.random.random(size=(n_topics, vocabulary_size))
    β /= np.sum(β, axis=1)[:, np.newaxis]
    if ρ is None:
        ρ = np.random.random(n_clusters)
        ρ /= np.sum(ρ)
    if π is None:
        pre_π = np.random.random((n_clusters, n_clusters))
        π = np.triu(pre_π, k=1) + np.triu(pre_π).T
    if θ is None:
        θ = np.random.dirichlet(alpha=α, size=(n_clusters, n_clusters))
    
    topics_range = range(n_topics)
    vocabulary_range = range(vocabulary_size)

    # Order Latent Variables
    Y = np.random.multinomial(1, pvals=ρ, size=n_nodes)
    A = np.random.binomial(1, Y @ π @ Y.T)
    Z = np.nan * np.zeros((n_nodes, n_nodes, n_documents, n_slots))
    W = np.nan * np.zeros((n_nodes, n_nodes, n_documents, n_slots))

    sources_A, destinations_A = np.where(A)
    for i, j in tqdm_notebook(zip(sources_A, destinations_A), "Z[i, j, :, :]"):
        Z[i, j, :, :] = np.random.choice(topics_range, p=θ[Y[i].astype(bool), Y[j].astype(bool)][0],
                                         size=(n_documents, n_slots))

    for k in tqdm_notebook(topics_range, "W[k_idx]"):
        k_idx = np.where(Z == k)
        W[k_idx] = np.random.choice(vocabulary_range, p=β[k], size=len(k_idx[0]))

    return (Y, A, Z, W)

In [3]:
## Scenario 1
print("### Creating scenario 1 ###")

n_clusters_S1 = 3
n_topics_S1 = 4

ρ_S1 = np.ones(n_clusters_S1) / n_clusters_S1
π_S1 = 0.24 * np.diag(np.ones(n_clusters_S1)) + 0.01
θ_S1 = np.zeros((n_clusters_S1, n_clusters_S1, n_topics_S1))
θ_S1[1, 1, 1], θ_S1[2, 2, 2], θ_S1[0, 0, 0] = 1, 1, 1

for i, j in tqdm_notebook(itertools.product(range(n_clusters_S1), range(n_clusters_S1)), "θ"):
    if i != j:
        θ_S1[i, j, 3] = 1

print('ρ_S1\n', ρ_S1)
print()
print('π_S1\n', π_S1)
print()
print('θ_S1\n', θ_S1)
print()
print()

Y_S1, A_S1, Z_S1, W_S1 = stbm(n_clusters=n_clusters_S1, n_topics=n_topics_S1,
                              ρ=ρ_S1, π=π_S1, θ=θ_S1)
lab_S1 = np.sum(W_S1, axis=(1, 2, 3))


## Scenario 2
print("### Creating scenario 2 ###")

n_clusters_S2 = 2
n_topics_S2 = 3

ρ_S2 = np.ones(n_clusters_S2) / n_clusters_S2
π_S2 = 0.25 * np.ones((n_clusters_S2, n_clusters_S2))
θ_S2 = np.zeros((n_clusters_S2, n_clusters_S2, n_topics_S2))
θ_S2[1, 1, 1], θ_S2[0, 0, 0] = 1, 1

for i, j in tqdm_notebook(itertools.product(range(n_clusters_S2), range(n_clusters_S2)), "θ"):
    if i != j:
        θ_S2[i, j, 2] = 1

print('ρ_S2\n', ρ_S2)
print()
print('π_S2\n', π_S2)
print()
print('θ_S2\n', θ_S2)
print()
print()

Y_S2, A_S2, Z_S2, W_S2 = stbm(n_clusters=n_clusters_S2, n_topics=n_topics_S2,
                              ρ=ρ_S2, π=π_S2, θ=θ_S2)
lab_S2 = np.sum(W_S2, axis=(1, 2, 3))


## Scenario 3
print("### Creating scenario 3 ###")

n_clusters_S3 = 4
n_topics_S3 = 3

ρ_S3 = np.ones(n_clusters_S3) / n_clusters_S3
π_S3 = 0.24 * np.diag(np.ones(n_clusters_S3)) + 0.01
θ_S3 = np.zeros((n_clusters_S3, n_clusters_S3, n_topics_S3))
θ_S3[2, 2, 0], θ_S3[0, 0, 0], θ_S3[1, 1, 1], θ_S3[3, 3, 1] = 1, 1, 1, 1

for i, j in tqdm_notebook(itertools.product(range(n_clusters_S3), range(n_clusters_S3)), "θ"):
    if i != j:
        θ_S3[i, j, 2] = 1

print('ρ_S3\n', ρ_S3)
print()
print('π_S3\n', π_S3)
print()
print('θ_S3\n', θ_S3)
print()
print()

Y_S3, A_S3, Z_S3, W_S3 = stbm(n_clusters=n_clusters_S3, n_topics=n_topics_S3,
                              ρ=ρ_S3, π=π_S3, θ=θ_S3)
lab_S3 = np.sum(W_S3, axis=(1, 2, 3))

### Creating scenario 1 ###



ρ_S1
 [0.33333333 0.33333333 0.33333333]

π_S1
 [[0.25 0.01 0.01]
 [0.01 0.25 0.01]
 [0.01 0.01 0.25]]

θ_S1
 [[[1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]

 [[0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]]

 [[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]]





### Creating scenario 2 ###



ρ_S2
 [0.5 0.5]

π_S2
 [[0.25 0.25]
 [0.25 0.25]]

θ_S2
 [[[1. 0. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 1. 0.]]]





### Creating scenario 3 ###



ρ_S3
 [0.25 0.25 0.25 0.25]

π_S3
 [[0.25 0.01 0.01 0.01]
 [0.01 0.25 0.01 0.01]
 [0.01 0.01 0.25 0.01]
 [0.01 0.01 0.01 0.25]]

θ_S3
 [[[1. 0. 0.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 1. 0.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [1. 0. 0.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 1. 0.]]]




In [4]:
np.savez(STBM_PATH + '/scenario-1.npz', Y=Y_S1, A=A_S1, Z=Z_S1, W=W_S1, lab=lab_S1)
np.savez(STBM_PATH + '/scenario-2.npz', Y=Y_S2, A=A_S2, Z=Z_S2, W=W_S2, lab=lab_S2)
np.savez(STBM_PATH + '/scenario-3.npz', Y=Y_S3, A=A_S3, Z=Z_S3, W=W_S3, lab=lab_S3)